In [0]:
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy import signal
%matplotlib inline

In [0]:
# diffusion rate of A
#dA = 2.8e-4
dA = 1
# diffusion rate of B
#dB = 5e-3
dB = 0.5
# kill rate
#k = -.005
k = .0567
# feed rate
f = .0649

# the Karl Sims numbers: DA=1.0, DB=.5, f=.055, k=.062

In [0]:
T = 64.0  # total time
dt = .001  # time step
n = int(T / dt)  # number of iterations

In [0]:
size = 100  # size of the 2D grid
dx = 2. / size  # space step

In [0]:
# this function returns a matrix that contains the laplacian of each cell using partial difference method
def laplacian(Z):
    Ztop = Z[0:-2, 1:-1]
    Zleft = Z[1:-1, 0:-2] # indexing to -2 gets you the second-to-last character
    Zbottom = Z[2:, 1:-1]
    Zright = Z[1:-1, 2:]
    Zcenter = Z[1:-1, 1:-1]
    return (Ztop + Zleft + Zbottom + Zright -
            4 * Zcenter) / dx**2

In [0]:
# karl sims laplacian function
# estimating Laplacian using 3x3 convolution
# weights for convolution: center weight -1, adjacent neighbors .2, and diagonals .05
def sims_laplacian(Z):
  weights = np.array([[ 0.05, 0.2,  0.05],
                       [0.2, -1.0, 0.2],
                       [ 0.05, 0.2,  0.05]])
  lap = signal.convolve2d(Z, weights, mode='same')
  return lap #[1:-1, 1:-1]

In [0]:
# display the grid
def show_patterns(U, ax=None):
    ax.imshow(U, cmap=plt.cm.copper,
              interpolation='bilinear',
              extent=[-1, 1, -1, 1])
    ax.set_axis_off()

In [0]:
# initializing the two "chemicals" randomly
A = np.random.rand(size, size)
B = np.random.rand(size, size)

In [0]:
# initializing the two chemicals in the Karl Simms way
A = np.ones((size, size))
B = np.zeros((size, size))
for i in range(10):
  x,y = random.randint(1,size-1), random.randint(1,size-1)
  B[x,y] = 1
  A[x,y] = 0

In [0]:
# another initialization method
A = np.ones((size, size))
B = np.zeros((size, size))
x,y = random.randint(1,size-1), random.randint(1,size-1)
B[x:x+int(size/4),y:y+int(size/5)] = 1
#A[x:x+int(size/4),y:y+int(size/5)] = 0

In [0]:

#for github version
tau = 0.1

fig, axes = plt.subplots(8, 8, figsize=(8, 8))
step_plot = n // 64

# What is the finite difference method for simulating PDEs?
# gives an explanation of how to use finite difference method, but does not give theory behind how it works: https://www.youtube.com/watch?v=bLiaz-IHX_Y&t=634s

# for each time step
for i in range(n):
  # calculate the laplacian for that square
  A_laplac = sims_laplacian(A)
  B_laplac = sims_laplacian(B)
  # store previous values of A and B in that square
  #A_prev = A[1:-1, 1:-1]
  #B_prev = B[1:-1, 1:-1]
  A_prev = A
  B_prev = B
  
  # update values of A and B using the mysterious github equation
  #A[1:-1, 1:-1], B[1:-1, 1:-1] = \
        #A_prev + dt * (dA * A_laplac + A_prev - A_prev**3 - B_prev + k),\
        #B_prev + dt * (dB * B_laplac + A_prev - B_prev) / tau
  
  # update the values of A and B using the Karl Simms equation
  #[1:-1, 1:-1]
  A = A_prev + (dA*A_laplac - A_prev*B_prev**2 + f*(1-A_prev))*dt
  B = B_prev + (dB*B_laplac + A_prev*B_prev**2 - (k+f)*B_prev)*dt
  
  # not adhering to Neumann conditions (derivatives at the edges are not null) --> this seems to cause everything to disperse to the edges
  # does it disperse to the edges because the derivatives are essentially infinite at those edges? 
  
  # set edges to zero --> this makes everything disappear --> actually everything disappeared because the dispersion rates were too high
  #A[0, :] = 0
  #A[:, 0] = 0
  #A[-1, :] = 0
  #A[:, -1] = 0
  #B[0, :] = 0
  #B[:, 0] = 0
  #B[-1, :] = 0
  #B[:, -1] = 0
  
  # set edge derivatives to null --> WHAT IS THE DIFFERENCE BETWEEN NULL AND ZERO?
  for Z in (A, B):
        Z[0, :] = Z[1, :]
        Z[-1, :] = Z[-2, :]
        Z[:, 0] = Z[:, 1]
        Z[:, -1] = Z[:, -2]
  
  
  # at certain intervals visualize the grid
  # We plot the state of the system at
  # 9 different times.
  if i % step_plot == 0 and i < 64 * step_plot:
  #if i == 50:
      ax = axes.flat[i // step_plot]
      show_patterns(A, ax=ax)
      ax.set_title(f'$t={i * dt:.2f}$')

In [0]:
# THE KARL-SIMS WAY --> I think the code above is basically a vectorized version of this
# laplacian function needs to become a 3x3 convolution
# for each time step
  # for each square in grid
    # calculate the laplacian of that square (I think it's a discreet laplacian but not really sure?) 
    # store previous A and B values of that square
    # update A and B values of that square
  # at certain time points, visualize the grid